# 推薦システム
このファイルは、ユーザーに合わせて好みの単語帳を予測し、各ユーザーへ推薦を行うためのモジュールです。
推薦アルゴリズムは、古典的な手法の「協調フィルタリング」を実装します（オフライン）。

## 手順
1. 各種ライブラリのimport
2. DBの接続
3. userテーブル取得
4. wordbookテーブル取得
5. 評価値行列の作成
6. 推薦アルゴリズムの作成 (ユーザーベース協調フィルタリング)
7. 次元削減 (PCA, NMF)
8. 6、7のモデル評価
9. 推薦モデルをpickleでオブジェクト化し、Djangoniへ組み込む

※ 一般的に3~6を「前処理」、7~8は「機械学習」と呼びます。

### 1. 各種ライブラリのimport

In [1]:
import os
import sqlite3
import psycopg2
import pickle
# recommender
import numpy as np
from scipy.spatial.distance import cosine
import pandas as pd
from sklearn.decomposition import PCA, NMF

### 2. DBの接続

In [12]:
# working directory path
working_dir = os.path.abspath('__file__')

# project directory path
project_dir = os.path.dirname \
    (os.path.dirname(os.path.dirname(working_dir)))

# db path
db_path = os.path.join(project_dir, 'db.sqlite3')

# connect db
if 'db.sqlite3' in os.listdir(project_dir):
    con = sqlite3.connect(db_path)  # sqlite3
else:
    con = psycopg2.connect(dbname='wordbook')  # PostgreSQL

### 3. userテーブル取得

In [13]:
# user
q_user = '''
SELECT
    *
FROM
    user AS u
'''

# user DataFrame
user = pd.read_sql(q_user, con)

# index
index = user['username']

index

0    admin
1     test
2     hoge
3    test2
4    test3
5      てすと
6     てすてす
Name: username, dtype: object

### 4. wordbookテーブル取得

In [14]:
# wordbook
q_wordbook = '''
SELECT
    *
FROM
    note AS n
'''

# wordbook DataFrame
wordbook = pd.read_sql(q_wordbook, con)

# columns
columns = wordbook['title']

columns

0                Django
1                  test
2                  hoge
3                  asdf
4                 vim基礎
5                SQLite
6           VScode 拡張機能
7         JavaScirpt 基礎
8           VScode コマンド
9                   AWS
10                test3
11                test2
12               Python
13         shellでファイル実行
14    マーケティング 入門（ちきりん）
15            推薦システム　入門
16          Mac ショートカット
Name: title, dtype: object

### 5. 評価値行列の作成

In [ ]:

U = '''
SELECT
    u.username
FROM
    user AS u
    INNER JOIN (
        SELECT
            *
        FROM
            star AS s
            INNER JOIN
                note AS n
                ON s.note_id == n.id
        ) AS s
        ON u.id == s.user_id
'''

V = '''
SELECT
    n.title
    , CASE WHEN 
FROM
    note AS n
    INNER JOIN
        star AS s
        ON n.id == s.note_id
'''

# DataFrame
# R = pd.read_sql(rating_matrix, con, index=index, columns=columns)

# df

### コサイン類似度

In [ ]:
def calc_distance(u_index, u_matrix):
    """ユーザベース協調フィルタリング"""

    # user_iの評価値ベクトル
    pd_type = pd.core.frame.DataFrame
    if type(u_matrix) == pd_type:
        ratings = u_matrix.iloc[u_index, :]
    else:
        ratings = u_matrix[u_index]
    # 最後の推薦で扱う、i`temと同じ次元数の推薦ベクトルを定義
    similarity = np.zeros(len(ratings))

    # user_iと、各userの評価値ベクトルからコサイン類似度を求める
    for c_index in range(len(u_matrix)):
        # user_cの評価値べクトル
        if type(u_matrix) == pd_type:
            compare_ratings = u_matrix.iloc[c_index, :]
        else:
            compare_ratings = u_matrix[c_index]
        # c_indexがuser_iと同じindexの場合、計算しない
        if u_index == c_index:
            continue
        # コサイン類似度
        cosine_simirarity = 1. - cosine(ratings, compare_ratings)
        # コサイン類似度をuser_cの評価値ベクトルと掛け合わせる
        user_ratings = cosine_simirarity * compare_ratings
        # 掛け合わせたモノを推薦ベクトルに足す
        similarity += user_ratings
    return similarity